In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in list(filenames)[:25]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import the dataset

In [ ]:
from tqdm import tqdm

In [ ]:
import json

In [ ]:
with open('/kaggle/input/coleridgeinitiative-show-us-the-data/test/8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60.json') as sample:
    sample_json = json.load(sample)

In [ ]:
sample_json[:3]

In [ ]:
# [{}, {}, {}]
for i in sample_json:
    print(i.keys())

In [ ]:
## 

## Preprocessing

In [ ]:
import nltk
from typing import List
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')

In [ ]:
def remove_stopwords(input_text: str, stopwords: List[str]) -> str:
    """Given an input text, removes all stopwords and punctuation marks"""
    text_tokens = word_tokenize(input_text)
    return [word.lower() for word in text_tokens if not word.lower() in stopwords and word.lower().isalnum()]

In [ ]:
remove_stopwords(sample_json[0]['text'], english_stopwords)[:10]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
sample_text = remove_stopwords(sample_json[0]['text'], english_stopwords)

vectorizer = CountVectorizer()

In [ ]:
corpus = [" ".join(remove_stopwords(text_body['text'] ,english_stopwords)) for text_body in sample_json]

In [ ]:
corpus[0][:100]

In [ ]:
X = vectorizer.fit_transform(corpus)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

# TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [ ]:
def get_string(filename) -> str:
    ...
#     Load file from memory
    with open(filename, 'r') as f:
        doc_json = json.load(f)
    body = ' '.join([dictionary['text'] for dictionary in doc_json])
    return ' '.join([word for word in body.split(' ') if (word.isalpha() and word.isascii())])

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words=english_stopwords, max_features=5000)

In [ ]:
train_files = []
for dirname, _, filenames in os.walk('/kaggle/input/coleridgeinitiative-show-us-the-data/train/'):
    for filename in filenames:
        if filename.endswith('.json'):
            train_files += [os.path.join(dirname, filename)]

# train_files = train_files[:1000]
            
doc_strings = [get_string(file_path) for file_path in tqdm(train_files)]
    
    
X = tfidf_vectorizer.fit_transform(doc_strings)


In [ ]:
tfidf_df = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names())
tfidf_df

In [ ]:
def calculate_similarity(X, vectorizor, query, top_k=20):
    """ Vectorizes the `query` via `vectorizor` and calculates the cosine similarity of
    the `query` and `X` (all the documents) and returns the `top_k` similar documents."""

    # Vectorize the query to the same length as documents
    query_vec = vectorizor.transform(query)
    # Compute the cosine similarity between query_vec and all the documents
    cosine_similarities = cosine_similarity(X,query_vec).flatten()
    # Sort the similar documents from the most similar to less similar and return the indices
    most_similar_doc_indices = np.argsort(cosine_similarities, axis=0)[:-top_k-1:-1]
    return (most_similar_doc_indices, cosine_similarities)

def show_similar_documents(df, cosine_similarities, similar_doc_indices, query):
    """ Prints the most similar documents using indices in the `similar_doc_indices` vector."""
    counter = 1
    for index in similar_doc_indices:
        print('Top-{}, Similarity = {}'.format(counter, cosine_similarities[index]))
        filename = train_files[index].split('/')[-1].split('.')[0]
        print(f"Filename: {filename}")
        cleaned_label = list(train[train['Id'] == filename]['cleaned_label'])
        print(f"cleaned_label: {cleaned_label}")
        print('body: {}, '.format(df[index])[:100])
        print()
        counter += 1
        if bool([c for c in cleaned_label if c == query])

In [ ]:
# query = ['national science foundation survey of industrial research and development']
# sim_vecs, cosine_similarities = calculate_similarity(X, tfidf_vectorizer, query)

for query in unique_dataset_labels:
    sim_vecs, cosine_similarities = calculate_similarity(X, tfidf_vectorizer, query)
    show_similar_documents(doc_strings, cosine_similarities, sim_vecs, query)

In [ ]:
train

## Train 130 different classifiers
One for each dataset label

In [ ]:
train = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')

In [ ]:
unique_dataset_labels = train['cleaned_label'].unique()
print(unique_dataset_labels)

## Load dataset values

In [ ]:
train_results = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')

In [ ]:
train_results.groupby('Id').count().sort_values(by='pub_title',ascending=False)

In [ ]:
train_results[train_results['Id']=='170113f9-399c-489e-ab53-2faf5c64c5bc']

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')

In [ ]:
with open('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv', 'r') as f:
    sample_submission = f.read()


## Submission format
| Id | PredictionString |
| --- | --- | 
| 2100032a-7c33-4bff-97ef-690822c43466 |adni|alzheimer s disease neuroimaging initiati |
| 2f392438-e215-4169-bebf-21ac4ff253e1 |common core of data\|nces common core of data|t |
| 3f316b38-1a24-45a9-8d8c-4e05a42257c6 |slosh model\|noaa storm surge inundation|sea la |
| 8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60 |rural urban continuum codes |

In [ ]:
print(sample_submission)

### Turn document into single string

In [ ]:
def get_string(filename) -> str:
    ...
#     Load file from memory
    with open(filename, 'r') as f:
        doc_json = json.load(f)
    return ' '.join([dictionary['text'] for dictionary in doc_json])
        

In [ ]:
get_string('/kaggle/input/coleridgeinitiative-show-us-the-data/test/8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60.json')[:100] + '...'

### Create set of all possible datasets

In [ ]:
train = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
print(len(train['dataset_title'].unique()))
print(len(train['dataset_label'].unique()))
print(len(train['cleaned_label'].unique()))
print(train.shape)

In [ ]:
list(train.groupby('dataset_title'))[0][1]

In [ ]:
def get_dataset_map() -> dict:
    label_map = {}

    for ndx, row in train.iterrows():
        if not label_map.get(row['dataset_label'],False):
            label_map[row['dataset_label']] = []

        if not label_map.get(row['dataset_title'],False):
            label_map[row['dataset_title']] = []

        label_map[row['dataset_label']] += [row['cleaned_label']]
        label_map[row['dataset_title']] += [row['cleaned_label']]

    for k, v in label_map.items():
        label_map[k] = list(set(v))

    return label_map

In [ ]:
label_map = get_dataset_map()
label_map

In [ ]:
def find_datasets_in_string(corpus: str, label_map: dict) -> list:
    predictions = []
    for k, v in label_map.items():
        if k in corpus:
            predictions+=v
    predictions_set = set(predictions)
    return '|'.join(predictions_set)
    
    
    

In [ ]:
sample_doc = get_string('/kaggle/input/coleridgeinitiative-show-us-the-data/test/8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60.json')
print(find_datasets_in_string(sample_doc, label_map))

In [ ]:
train[train['Id'] =='8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60']

### Search Document for all datasets in set

In [ ]:
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('.json'):
            files += [os.path.join(dirname, filename)]

In [ ]:
results = {}
for doc in tqdm(files):
    results[doc.split('/')[-1].split('.')[0]] = find_datasets_in_string(get_string(doc), label_map)

In [ ]:
list(results.items())[:4]

## Format Submission

In [ ]:
get_csv_df(results)

## Create Test Set Submission

In [ ]:
files = []
for dirname, _, filenames in os.walk('/kaggle/input/coleridgeinitiative-show-us-the-data/test/'):
    for filename in filenames:
        if filename.endswith('.json'):
            files += [os.path.join(dirname, filename)]
results = {}
for doc in tqdm(files):
    results[doc.split('/')[-1].split('.')[0]] = find_datasets_in_string(get_string(doc), label_map)

In [ ]:
submission_df = get_csv_df(results)
submission_df

In [ ]:
submission_df.to_csv('submission.csv',index=False)

In [ ]:
with open('submission.csv' , 'r') as f:
    print(f.read())

## Modeling